<a href="https://colab.research.google.com/github/YoheiFukuhara/keras-for-beginner/blob/master/Keras02_SaveModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[「【入門者向け解説】Kerasで単純なディープラーニングモデル定義」](https://qiita.com/FukuharaYohei/items/579cc0184885b4ba75df)で解説しています。

ブログ記事[「無から始めるKeras」](https://qiita.com/Ishotihadus/items/c2f864c0cde3d17b7efb)のシリーズおよび[Kerasによる、ものすごくシンプルな深層学習の例](https://qiita.com/hiroeorz@github/items/33b85529be0829f34973)を参考にしました。


In [0]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import ModelCheckpoint, Callback

# テストデータの作成


## 説明変数
0から1までの乱数で128, 2の配列を作成

In [0]:
NUM_TRAIN = 128
data = np.random.rand(NUM_TRAIN,2)

## 目的変数
説明変数の各行の2つの値を足した数

In [0]:
labels = (np.sum(data, axis=1) > 1.0) * 1
labels = labels.reshape(NUM_TRAIN,1)

# モデル定義

In [9]:
# Sequentialモデル使用(Sequentialモデルはレイヤを順に重ねたモデル)
model = Sequential()

# 全結合層(2層->4層)
model.add(Dense(4, input_dim=2, activation="tanh"))

# 結合層(4層->1層)：入力次元を省略すると自動的に前の層の出力次元数を引き継ぐ
model.add(Dense(1, activation="sigmoid"))

# モデルをコンパイル
model.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 4)                 12        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 17
Trainable params: 17
Non-trainable params: 0
_________________________________________________________________


#訓練

In [0]:
callbacks = []
callbacks.append(ModelCheckpoint('./model.hdf5', save_best_only=True))

In [11]:
model.fit(data, labels, nb_epoch=300, validation_split=0.2, callbacks=callbacks)

Train on 102 samples, validate on 26 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/300
102/102 [==============================] - 0s 5ms/sample - loss: 0.6815 - acc: 0.4902 - val_loss: 0.6120 - val_acc: 0.5769
Epoch 2/300
102/102 [==============================] - 0s 250us/sample - loss: 0.6766 - acc: 0.4902 - val_loss: 0.6101 - val_acc: 0.5769
Epoch 3/300
102/102 [==============================] - 0s 260us/sample - loss: 0.6736 - acc: 0.4902 - val_loss: 0.6083 - val_acc: 0.5769
Epoch 4/300
102/102 [==============================] - 0s 265us/sample - loss: 0.6703 - acc: 0.5000 - val_loss: 0.6062 - val_acc: 0.5769
Epoch 5/300
102/102 [==============================] - 0s 257us/sample - loss: 0.6668 - acc: 0.5098 - val_loss: 0.6040 - val_acc: 0.5769
Epoch 6/300
102/102 [==============================] - 0s 258us/sample - loss: 0.6627 - acc: 0.5098 - val_loss: 0.6017 - val_acc: 0.6154
Epoch 7/300
102/102 [==============================] - 0s 288us/sample - loss: 0.6585 -

# テスト実行

In [0]:
NUM_TEST = 50
test_data = np.random.rand(NUM_TEST,2)
test_labels = (np.sum(test_data, axis=1) > 1.0) * 1

In [0]:
predict = ((model.predict(test_data) > 0.5) * 1).reshape(NUM_TEST)
print(predict)
print(test_labels)
print("Accuracy:",sum(predict == test_labels) / NUM_TEST)

[1 1 1 0 0 1 1 1 1 0 0 1 0 0 1 1 0 1 1 0 1 0 0 0 1 1 0 1 1 1 0 0 1 1 1 0 1
 1 1 1 1 0 1 1 0 0 1 0 1 0]
[0 1 1 0 0 1 1 0 1 0 0 1 0 0 1 1 0 0 1 0 1 0 0 0 1 1 0 1 1 1 0 0 0 1 1 0 1
 1 1 1 1 0 1 0 0 0 1 0 1 0]
Accuracy: 0.9


In [0]:
# ライブラリバージョン確認
!pip freeze

absl-py==0.7.1
alabaster==0.7.12
albumentations==0.1.12
altair==3.0.1
astor==0.7.1
astropy==3.0.5
atari-py==0.1.7
atomicwrites==1.3.0
attrs==19.1.0
audioread==2.1.6
autograd==1.2
Babel==2.6.0
backcall==0.1.0
backports.tempfile==1.0
backports.weakref==1.0.post1
beautifulsoup4==4.6.3
bleach==3.1.0
bokeh==1.0.4
boto==2.49.0
boto3==1.9.148
botocore==1.12.148
Bottleneck==1.2.1
branca==0.3.1
bs4==0.0.1
bz2file==0.98
cachetools==3.1.0
certifi==2019.3.9
cffi==1.12.3
chainer==5.4.0
chardet==3.0.4
Click==7.0
cloudpickle==0.6.1
cmake==3.12.0
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.1.3
coverage==3.7.1
coveralls==0.5
crcmod==1.7
cufflinks==0.14.6
cvxopt==1.2.3
cvxpy==1.0.15
cycler==0.10.0
cymem==2.0.2
Cython==0.29.7
cytoolz==0.9.0.1
daft==0.0.4
dask==1.1.5
dataclasses==0.6
datascience==0.10.6
decorator==4.4.0
defusedxml==0.6.0
dill==0.2.9
distributed==1.25.3
Django==2.2.1
dlib==19.16.0
dm-sonnet==1.32
docopt==0.6.2
docutils==0.14
dopamine-rl==1.0.5
easydict==1.9
ecos=